In [50]:
import scanpy as sc
import dynamo as dyn 
import spateo as spa 
import pyvista as pv
pv.set_jupyter_backend('panel')

In [2]:
adata = sc.read('/lab/solexa_weissman/xqiu/proj/Aristotle/spateo_project/revision/data/3D_reconstruction/E16.5_3D.test.5section.h5ad')

adata

Only considering the two last: ['.5section', '.h5ad'].
Only considering the two last: ['.5section', '.h5ad'].


AnnData object with n_obs × n_vars = 603481 × 27212
    obs: 'coor_x', 'coor_y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'spatial_leiden_res1', 'spatial_leiden_res2', 'spatial_leiden_res3', 'plot1', 'sample', 'batch', 'order'
    var: 'n_cells-0', 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'n_cells-1', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'n_cells-2', 'n_cells_by_counts-2', 'mean_counts-2', 'log1p_mean_counts-2', 'pct_dropout_by_counts-2', 'total_counts-2', 'log1p_total_counts-2', 'n_cells-3', 'n_cells_by_counts-3', 'mean_counts-3', 'log1p_mean_counts-3', 'pct_dropout_by_counts-3', 'total_counts-3', 'log1p_total_counts-3', 'n_cells-4', 'n_

5 sections of E16.5 embryo.
1. Aligned coordinate is in adata.obsm['spatial_align'], the third value is the Z axis. The value of X, Y, Z axis are in same scale.
2. Order of sections is in adata.obs['order'], though the Z axis could reflect the order as well

1) learn the 3D expression model; 
2). plot genes that show polarity; 
3). plot genes changes over Z axis

In [3]:
# atrium and ventricle of heart

genes = ["Myl7", "Actc1", "Myl4", "Tnnt2", "Tpm1", "Tnni1", "Tnnc1", "Myl3", "Myl2", "Tnni3", "Csrp3", "Ttn", "Myh6", "Myh7", "Myl9", "Cox6a2", "Cryab"]


# sparseVFC model

In [4]:
import numpy as np
from dynamo.vectorfield.scVectorField import SparseVFC


def interpolation_SparseVFC(adata, genes=None, grid_num=50, lambda_=0.02, lstsq_method="scipy", **kwargs):
    """
    predict missing location’s gene expression and learn a continuous gene expression pattern over space
    Parameters
    ----------
        adata: :class:`~anndata.AnnData`
            AnnData object that contains spatial (numpy.ndarray) in the `obsm` attribute.
        genes: `list` (default None)
            Gene list that needs to interpolate.
        grid_num: 'int' (default 50)
            Number of grid to generate. Default is 50. Must be non-negative.
        lambda_: 'float' (default: 0.02)
            Represents the trade-off between the goodness of data fit and regularization. Larger Lambda_ put more weights
            on regularization.
        lstsq_method: 'str' (default: `scipy`)
           The name of the linear least square solver, can be either 'scipy` or `douin`.
        **kwargs：
        Additional parameters that will be passed to SparseVFC function.
    Returns
    -------
    Res: 'dict'
        A dictionary which contains:
            X: Current location.
            valid_ind: The indices of cells that have finite expression values.
            X_ctrl: Sample control points of current location.
            ctrl_idx: Indices for the sampled control points.
            Y: expression estimates in delta t.
            beta: Parameter of the Gaussian Kernel for the kernel matrix (Gram matrix).
            V: Prediction of expression of X.
            C: Finite set of the coefficients for the
            P: Posterior probability Matrix of inliers.
            VFCIndex: Indexes of inliers found by sparseVFC.
            sigma2: Energy change rate.
            grid: Grid of current location.
            grid_V: Prediction of expression of the grid.
            iteration: Number of the last iteration.
            tecr_vec: Vector of relative energy changes rate comparing to previous step.
            E_traj: Vector of energy at each iteration,
        where V = f(X), P is the posterior probability and VFCIndex is the indexes of inliers found by sparseVFC.
        Note that V = `con_K(Grid, X_ctrl, beta).dot(C)` gives the prediction of expression on Grid (but can also be any
        point in the gene expression location space).
    """

    X, V = adata.obsm["spatial"], adata[:, genes].X

    # Generate grid
    min_vec, max_vec = (
        X.min(0),
        X.max(0),
    )
    min_vec = min_vec - 0.01 * np.abs(max_vec - min_vec)
    max_vec = max_vec + 0.01 * np.abs(max_vec - min_vec)
    Grid_list = np.meshgrid(*[np.linspace(i, j, grid_num) for i, j in zip(min_vec, max_vec)])
    Grid = np.array([i.flatten() for i in Grid_list]).T

    res = SparseVFC(X, V, Grid, lambda_=lambda_, lstsq_method=lstsq_method, **kwargs)

    return res

In [5]:
adata

AnnData object with n_obs × n_vars = 603481 × 27212
    obs: 'coor_x', 'coor_y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'spatial_leiden_res1', 'spatial_leiden_res2', 'spatial_leiden_res3', 'plot1', 'sample', 'batch', 'order'
    var: 'n_cells-0', 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'n_cells-1', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'n_cells-2', 'n_cells_by_counts-2', 'mean_counts-2', 'log1p_mean_counts-2', 'pct_dropout_by_counts-2', 'total_counts-2', 'log1p_total_counts-2', 'n_cells-3', 'n_cells_by_counts-3', 'mean_counts-3', 'log1p_mean_counts-3', 'pct_dropout_by_counts-3', 'total_counts-3', 'log1p_total_counts-3', 'n_cells-4', 'n_

In [6]:
def sample_by_velocity(V, n, seed=19491001):
    np.random.seed(seed)
    tmp_V = np.linalg.norm(V, axis=1)
    p = tmp_V / np.sum(tmp_V)
    idx = np.random.choice(np.arange(len(V)), size=n, p=p, replace=False)
    return idx

In [32]:
grid_num = [50, 50, 200] 
lambda_=0.02
lstsq_method="scipy"

X, V = adata.obsm["spatial_align"], adata[:, genes].X.A

# Generate grid
min_vec, max_vec = (
    X.min(0),
    X.max(0),
)
min_vec = min_vec - 0.01 * np.abs(max_vec - min_vec)
max_vec = max_vec + 0.01 * np.abs(max_vec - min_vec)
Grid_list = np.meshgrid(*[np.linspace(i, j, k) for i, j, k in zip(min_vec, max_vec, grid_num)])
Grid = np.array([i.flatten() for i in Grid_list]).T

res = SparseVFC(X, V, Grid, lambda_=lambda_, lstsq_method=lstsq_method)


|-----> [SparseVFC] begins...
|-----> Sampling control points based on data velocity magnitude...
|-----> [SparseVFC] in progress: 100.0000%
|-----> [SparseVFC] finished [52.6530s]


In [8]:
res.keys()

dict_keys(['X', 'valid_ind', 'X_ctrl', 'ctrl_idx', 'Y', 'beta', 'V', 'C', 'P', 'VFCIndex', 'sigma2', 'grid', 'grid_V', 'iteration', 'tecr_traj', 'E_traj'])

In [9]:
res['X'].shape, res['grid'].shape, res['grid_V'].shape

((603481, 3), (15625, 3), (15625, 17))

In [10]:
# find 3D grid points close to the original x, y, coordinates 
# find 3D grid points within the Mesh of the convex formed by the data 

# scipy's ConvexHull to be 500X faster than using vtkDelaunay3D and vtkDataSetSurfaceFilter because you skip the expensive 3D tesselation of the volume.

In [34]:
import numpy as np
from scipy.spatial import ConvexHull
import pyvista as pv 
pv.start_xvfb()
pv.set_jupyter_backend('ipyvtklink')  
from pyvista import PolyData

def polyhull(x, y, z):
    hull = ConvexHull(np.column_stack((x, y, z)))
    faces = np.column_stack((3*np.ones((len(hull.simplices), 1), dtype=np.int), hull.simplices)).flatten()
    poly = PolyData(hull.points, faces)
    return poly
    
x, y, z = X[:, 0], X[:, 1], X[:, 2]

hull = polyhull(x, y, z)
hull.plot()

/home/xqiu/python37test-env/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [35]:
hull = ConvexHull(np.column_stack((x, y, z)))


In [36]:
hull.points

array([[-65.94364929, -14.57385921,  86.        ],
       [-66.09521484, -12.83043575,  86.        ],
       [-65.56467438, -16.04656982,  86.        ],
       ...,
       [-65.97429657,  -6.62344885, 122.        ],
       [-66.09428406,  -7.89254427, 122.        ],
       [-65.70823669, -11.62262058, 122.        ]])

In [37]:
hull.points.shape

(603481, 3)

In [38]:
hull.vertices

array([     0,      1,     70,    144,    145,    237,    436,    877,
         1003,   1413,   1557,   2330,   3183,   3722,   5038,   6032,
        14813,  15278,  15973,  22429,  31228,  32661,  35316,  35610,
        36206,  40680,  76132,  78219,  80880,  81673,  82969,  83214,
        83464,  86245,  88855,  89630,  92055,  93511,  96957,  97758,
        99817, 100581, 102591, 106931, 107389, 111002, 112582, 113885,
       114928, 115254, 116582, 117112, 117350, 117351, 117695, 117696,
       117697, 118025, 122384, 140494, 141090, 141688, 199548, 208923,
       212760, 213633, 216212, 236428, 236447, 237195, 237272, 237669,
       237764, 237868, 237980, 237981, 238092, 238186, 238273, 238366,
       238474, 238580, 238955, 240079, 240961, 242173, 243099, 243135,
       243136, 243358, 243359, 243445, 243727, 244178, 245395, 246026,
       246525, 251070, 261260, 266293, 280946, 333169, 333795, 334422,
       335981, 337221, 347721, 364064, 365681, 366264, 367915, 368090,
      

In [39]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    from scipy.spatial import Delaunay
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

grid_in_hull = in_hull(res['grid'], hull.points[hull.vertices, :])
grid_in_hull.sum()

321846

In [55]:
# res['grid'][:100]

In [ ]:
from scipy.spatial import Delaunay
tmp = Delaunay(hull.points)

In [51]:
type(grid_in_hull)

numpy.ndarray

In [40]:
from anndata import AnnData

In [41]:
intep_adata = AnnData(X=res['grid_V'][grid_in_hull], obsm={"spatial": res['grid'][grid_in_hull]}, var=adata[:, genes].var)

In [42]:
res['grid'].shape

(500000, 3)

In [43]:
intep_adata

AnnData object with n_obs × n_vars = 321846 × 17
    var: 'n_cells-0', 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'n_cells-1', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'n_cells-2', 'n_cells_by_counts-2', 'mean_counts-2', 'log1p_mean_counts-2', 'pct_dropout_by_counts-2', 'total_counts-2', 'log1p_total_counts-2', 'n_cells-3', 'n_cells_by_counts-3', 'mean_counts-3', 'log1p_mean_counts-3', 'pct_dropout_by_counts-3', 'total_counts-3', 'log1p_total_counts-3', 'n_cells-4', 'n_cells_by_counts-4', 'mean_counts-4', 'log1p_mean_counts-4', 'pct_dropout_by_counts-4', 'total_counts-4', 'log1p_total_counts-4'
    obsm: 'spatial'

In [44]:
adata.obs['order'].unique()

['1', '2', '3', '4', '5']
Categories (5, object): ['1', '2', '3', '4', '5']

# plot with pyvista 

In [45]:
intep_adata.obs['x'], intep_adata.obs['y'], intep_adata.obs['z'] = res['grid'][grid_in_hull, 0], res['grid'][grid_in_hull, 1], res['grid'][grid_in_hull, 2]

intep_adata.obs['cluster'] = 0 
from spateo.plotting.static.three_d_plots import recon_3d 
recon_3d(intep_adata,
                cluster = 'cluster',
                save = "3d.gif",
                cluster_show = "all",
                gene_show = genes[0],
                show = 'gene',
             )

In [27]:
?recon_3d

Signature:
recon_3d(
    adata: anndata._core.anndata.AnnData,
    cluster: str = 'cluster',
    save: str = '3d.png',
    cluster_show: Union[str, list] = 'all',
    gene_show: Union[str, list] = 'all',
    show: str = 'cluster',
    colormap: str = 'RdYlBu_r',
    background_color: str = 'black',
    other_color: str = 'white',
    off_screen: bool = True,
    window_size: Union[List[int], NoneType] = None,
    cpos: Union[list, NoneType] = None,
    bar_position: Union[list, NoneType] = None,
    bar_height: float = 0.3,
    viewup: Union[list, NoneType] = None,
    framerate: int = 15,
)
Docstring:
Draw a 3D image that integrates all the slices through pyvista,
and you can output a png image file, or a gif image file, or an MP4 video file.

Parameters
----------
adata : :class:`~anndata.AnnData`
    an Annodata object.
cluster : `str`
    Column name in .obs DataFrame that stores clustering results.
save : `str`
    If a str, save the figure. Infer the file type if ending on
    {'

# plot expression with 3D scatters 
- can also plot with meshgrid 

In [ ]:
# plot expression along the z-axis 


# deep learning model 

In [49]:
from pyvista import demos
demos.plot_logo(jupyter_backend='pythreejs')

Renderer(camera=PerspectiveCamera(aspect=2.4, children=(DirectionalLight(color='#fefefe', intensity=0.25, posi…

In [48]:
from pyvista import examples
mesh = examples.download_lucy()
mesh.plot(color='lightgrey', pbr=True, metallic=0.2,
          jupyter_backend='panel')